# Control of a 3D Quadcopter

The aim of this project is to develop a controller for a 3D quadcopter. The [starter Code](https://github.com/udacity/FCND-Controls-CPP) of the project and the project [rubics](https://review.udacity.com/#!/rubrics/1643/view) are given in the links. This writeup of the project explains how each point in the project rubic is addressed in the project.

## Control Architecture

A quadcopter is an under-actuated system. It has 6 degrees of freedom (3 - translation and 3 - rotations) but it has only 4 acutuators (rotors). Another challenge for controlling a quadcopter is the coupling of motions i.e. one degree of freedom depends on state of another degree of freedom. For example the following equations govern the motion of a quadcopter in 2D plane.

### 2D - Dynamics of Quadcopter

<img src="Pictures\Drone_2d.png" width="300">

$$\begin{align}
\ddot{z} &= g - \frac{u_1}{m}.cos\phi
\\
\\
\ddot{y} &= \frac{u_1}{m}.sin\phi
\\
\\
\ddot{\phi} &= \frac{u_2}{I_{xx}}
\end{align}$$

$$\begin{equation}
\\
where
\\
u_1 = F_1 + F_2
\\
u_2 = M_x = (F_1 - F_2).l
\\
\end{equation}$$

By linearing the equations the control problem can be simplified.Linearized equations follows:

$$\begin{align}
\ddot{z} &= g - \frac{u_1}{m}
\\
\\
\ddot{y} &= g \phi
\\
\\
\ddot{\phi} &= \frac{u_2}{I_x}
\end{align}$$

From the linearized equations it can be seen that the actuation command $u_1$ position can be used to control $z$ and the actuation command $u_2$ can be used to control either $\phi$ oreintation or $y$ position. The control $y$ position cannot be done directly but it is done via $\phi$. The dynamics of $z$ and $\phi$ motion are second order systems.But the dynamics of the $y$ motion is a fourth order system since it is controlled indirectly via $\phi$.

The $y$ position is controlled using a cascaced control loop(nested control loop) .The inner control loop is used to control the $\phi$ position and the outer loop controls the $y$ position. The following picture shows the cascaded structure of the $y$ motion control loop.

<img src="Pictures\Position_Control_2d.png">

### Moving to 3D 

Good understanding the dynamics of the quadcopter is 2D is very helpful when designing the controller for 3D motion of the quadcopter. The 3D dynamics of the quadcopter is discussed in detail the lecture. The 3D dyamic model, and control architecture discussed in the lecture is used in the project. The following picture shows the control architecture for the 3D - controller.

<img src="Pictures\control1.png" width="600">
The attitude controller breaks down into smaller controllers responsible for roll-pitch, yaw, and body rate. 
<img src="Pictures\control2.png" width="400">


The 3D control architecture is similar to the 2D control architecture. The $z$ movement and $\psi$ are controlled invidually using separate control loops of second order. The control of $y$ and $x$ are done using $\phi$ and $\theta$ using a cascaded control loop structure. The control loop involving - Position controller,Roll-pitch controller and body rate controller form the cascaded control loop for position control. The controllers yaw controller and body rate controller are used to control yaw angle. The altitude controller controls the $z$ direction movement. The difference in between 2D and 3D motion lies in the Roll-Pitch controller, where the Roll,Pitch angluar velocities($\dot{\phi},\dot{\theta}$) are calcuated in intertial frame and then converted to $\dot{p},\dot{q}$ angular velocities  in the body frame.

All the control loops in the controller uses cascaded controller instead of using PD controller. In a second order cascaded controller the output of each integrator is feedback and controlled by a propotinal gain. The following picture shows the structure of second order cascade controller.

<img src="Pictures\SecondOrderCascadeControl.png">

The tranfer funciton of the cascaded controller is given by

$G(s)=\frac{K_V.K_V}{S^2+K_V.S+K_V.K_P}=\frac{\omega_0^2}{S^2+2.D.\omega_0.S+\omega_0^2}$

So

$K_V = 2.D.\omega_0, K_P = \frac{\omega_0}{2.D}$

$\frac{K_V}{K_P}=4D^2$

Therefore for a damping ratio between 0.7 to 1 the ratio $\frac{K_V}{K_P}$ is between 2 - 4.

## Altitude Controller

The altitude controller uses a second order cascaded controller to control the position of the quadcopter in $z$ direction. It is implemented in the AltitudeControl() method of the QuadControl class. The altitude controller calcualtes the error and the quadcopter total thrust in the intertial frame. Then the trust in interial frame is converted in to body frame using the rotation matrix and returned. This thrust is passed to the drone by calculating the individual rotor thrust. The total thrust is also used by roll-pitch controller. The following equation are implemented in the altitude contorller.

Linear acceleration can be expressed by the next linear equation
$$
\begin{pmatrix} \ddot{x} \\ \ddot{y} \\ \ddot{z}\end{pmatrix}  = \begin{pmatrix} 0 \\ 0 \\ g\end{pmatrix} + R \begin{pmatrix} 0 \\ 0 \\ c \end{pmatrix} 
$$

where $R = R(\psi) \times R(\theta) \times R(\phi)$. The individual linear acceleration has the form of 

$$
\begin{align}
\ddot{x} &= c b^x \\ 
\ddot{y} &= c b^y \\ 
\ddot{z} &= c b^z +g
\end{align}
$$ 
where $b^x = R_{13}$, $b^y= R_{23}$ and $b^z = R_{33}$ are the elements of the last column of the rotation matrix. 

We are controlling the vertical acceleration: 

$$\bar{u}_1 = \ddot{z} = c b^z +g$$ 

Therefore 

$$c = (\bar{u}_1-g)/b^z$$  


In this exercise a PD controller is used for the altitude which results in: 

$$\bar{u}_1 = k_{p-z}(z_{t} - z_{a}) + k_{d-z}(\dot{z}_{t} - \dot{z}_{a}) + \ddot{z}_t$$

It has to be noted that when tuning the altitude controller the ratio $\frac{K_V}{K_P}$ was set to 0.35 aproximately. Only this ratio made the controller stable and acheive good response time. High values for the velocity gain made the drone unstable.

## Attitude Controller

The body rate controller, yaw controller and the roll-pitch contorller form the attitude controller. Attitude controller controller controls the orientation($\phi,\theta,\psi$) of the drone in the intertial frame. The attitude controller is of second order cascaded structure.The body rate contoller is the innermost loop which controls the angular velocities($p,q,r$) of the drone in the body frame using the angular accerations in body frame($\dot{p},\dot{q},\dot{r}$).The angular velocity set points are given by the pitch-roll controller and the yaw controller. The drone oreintation angles with respect to $x,y-$axis in the inertial frame are coupled to each other so they are controlled together in the roll-pitch controller and yaw is controlled seperately in the yaw controller. The yaw angle in the intertial frame is the same as the yaw angle in the body frame and so no transformation is done in the yaw controller. In case of the roll-pitch controller the orientation error values and the commanded angular velocities are calculated in the inertial frame and converted to body frame using the rotational matrix. The following equations are implemented in the attitude controller.

### Body rate controller
The commanded roll, pitch, and yaw are collected by the body rate controller, and they are translated into the desired rotational accelerations along the axis in the body frame. 

$p_{\text{error}} = p_c - p$

$\bar{u}_p= k_{p-p} p_{\text{error}}$

$q_{\text{error}} = q_c - q$

$\bar{u}_q= k_{p-q} q_{\text{error}}$

$r_{\text{error}} = r_c - r$

$\bar{u}_r= k_{p-r} r_{\text{error}}$

### Roll-Pitch controller

The roll-pitch controller is a P controller responsible for commanding the roll and pitch rates ($p_c$ and $q_c$) in the body frame.  First, it sets the desired rate of change of the given matrix elements using a P controller. 

**Note** - subscript c means "commanded" and a means "actual"
$$
\begin{align}
\ddot{x} &= c b^x \\ 
\ddot{y} &= c b^y \\ 
\ddot{z} &= c b^z +g
\end{align}
$$ 


$$\dot{b}^x_c  = k_p(b^x_c - b^x_a)$$

$$\dot{b}^y_c  = k_p(b^y_c - b^y_a)$$

where $b^x_a = R_{13}$ and $b^y_a = R_{23}$. The given values can be converted into the angular velocities into the body frame by the next matrix multiplication. 

$$
\begin{pmatrix} p_c \\ q_c \\ \end{pmatrix}  = \frac{1}{R_{33}}\begin{pmatrix} R_{21} & -R_{11} \\ R_{22} & -R_{12} \end{pmatrix} \times \begin{pmatrix} \dot{b}^x_c \\ \dot{b}^y_c  \end{pmatrix} 
$$

### Yaw controller

Control over yaw is decoupled from the other directions. A P controller is used to control the drone's yaw.

$$r_c = k_p (\psi_t - \psi_a)$$


In the attitude controller the ratio $\frac{k_v}{k_p}$ is set to 3.33 approximately which makes the controller stable and quick response time.

## Lateral Position Controller

The lateral position controller is responsible for setting the lateral acceration commands $\ddot{x_c},\ddot{y_c}$ based on the error in position and velocities in the $x,y$ directions. The following equations are implemented in this controller.

$$
\begin{align}
\ddot{x}_{\text{command}} &=  k^x_p(x_t-x_a) + k_d^x(\dot{x}_t - \dot{x}_a)+ \ddot{x}_t \\
\ddot{y}_{\text{command}} &=  k^x_p(y_t-y_a) + k_d^x(\dot{y}_t - \dot{y}_a)+ \ddot{y}_t \\
\end{align}
$$


It has to be noted that when tuning the altitude controller the ratio $\frac{K_V}{K_P}$ was set to 0.35 aproximately. Only this ratio made the controller stable and acheive good response time. High values for the velocity gain made the drone unstable.

## Motor Thrust Calculation

The altitude and attitude controllers provide the total thrust $c$ and the moments in $\tau_x,\tau_y,\tau_z$ in the body frame of the drone. These forces and moments have to be converted to individual model thrust commands. The individual motor thrust can be calculated by solving the equation of forces along z-direction, moments in x,y,z direction. The following picture shows the layout the drone.

<div class="container" style="width: 100%;">
 <div class="D1 col-sm-6" style="width: 40%;">
   <img src="Pictures\Drone1.png" height="100">
 </div>
 <div class="D2 col-sm-6" style="width: 50%;">
   <img src="Pictures\Drone2.png" height="300">
 </div>
</div>

The following equation shows the forces and moments on the drone.


$$ F_1 + F_2 + F_3 + F_4 = F_t = c.m $$

$$ F_1 - F_2 + F_3 - F_4 = \frac{\tau_x}{l} $$

$$ F_1 + F_2 - F_3 - F_4 = \frac{\tau_y}{l} $$

$$ F_1 - F_2 - F_3 + F_4 = -\frac{\tau_z}{kappa} $$

This can be re - writtern into 

$$
\begin{pmatrix} 1 & 1 & 1 & 1 \\ 1 & -1 & -1 & 1 \\ 1 & 1 & -1 & -1\\ 1 & -1 & 1 & -1 \end{pmatrix} \times \begin{pmatrix} F_1 \\ F_2 \\ F_3\\ F_4 \end{pmatrix} = \begin{pmatrix} F_t \\ \frac{\tau_x}{l} \\ \frac{\tau_y}{l} \\ -\frac{\tau_z}{kappa} \end{pmatrix}
$$

Solving the avobe set of equations gives

$$
 \begin{pmatrix} F_1 \\ F_2 \\ F_3\\ F_4 \end{pmatrix} = \begin{pmatrix} 1 & 1 & 1 & 1 \\ 1 & -1 & 1 & -1 \\ 1 & 1 & -1 & -1\\ 1 & -1 & -1 & 1 \end{pmatrix} \times \begin{pmatrix} F_t \\ \frac{\tau_x}{l} \\ \frac{\tau_y}{l} \\ -\frac{\tau_z}{kappa} \end{pmatrix}
$$

## Conclusion

The controller designed sucessfully passes all the scenarios in the simulator.